In [1]:
# Nativos
import random as rn
import os
import sys
import gc

#calculo
import numpy as np
import pandas as pd
import scipy as sp

#grafico
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display
%matplotlib inline
sns.set(style="whitegrid")

#warning ignore future
import warnings
# warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings("ignore")
gc.collect()

BASE_DIR = os.path.dirname(os.getcwd())
if BASE_DIR not in sys.path: sys.path.append(BASE_DIR)

#from utils import *
#from graphs import *

SEED = 29082013
os.environ['PYTHONHASHSEED']=str(SEED)
np.random.seed(SEED)
rn.seed(SEED)

subfolder = "data"
os.listdir(subfolder)

['.ipynb_checkpoints',
 'CONTENT_CATEGORY.zip',
 'CONTENT_CATEGORY_BOTTOM.zip',
 'CONTENT_CATEGORY_TOP.zip',
 'conversiones.zip',
 'data.zip',
 'data2.zip',
 'device_data.zip',
 'PAGE.zip',
 'pageviews.zip',
 'pageviews_complemento.zip',
 'pageview_with_previous',
 'pageview_with_previous_2.csv',
 'sampleSubmission.zip',
 'SITE_ID.zip']

## LOAD PAGE VIEWS INFO

In [2]:
col_user = 'USER_ID'
pageview = pd.read_csv(
    "{}/pageview_with_previous_2.csv".format(subfolder), parse_dates=["FEC_EVENT"], dtype=str
).sort_values(
    [col_user, "FEC_EVENT"]
)
pageview['USER_ID'] = pageview['USER_ID'].astype(int)
#pageview['PAGE_PREVIOUS'] = pageview['PAGE_PREVIOUS'].apply(lambda _: _.split('_')[0])
print(pageview.shape)

pageview.head()

(22870354, 10)


,FEC_EVENT,PAGE,CONTENT_CATEGORY,SITE_ID,ON_SITE_SEARCH_TERM,USER_ID,weekday,schedule,tipo_dias,trimestre
1354,2018-01-01 09:56:47,none_14,4_2,2,1,0,0,trabajo_manana,7_12,1
1357,2018-01-01 09:57:19,14_14,4_2,2,1,0,0,trabajo_manana,7_12,1
1361,2018-01-01 09:57:48,14_10,4_2,2,1,0,0,trabajo_manana,7_12,1
1362,2018-01-01 09:57:48,10_12,4_2,2,1,0,0,trabajo_manana,7_12,1
1363,2018-01-01 09:57:49,12_11,4_2,2,1,0,0,trabajo_manana,7_12,1


In [3]:
pageview.dtypes

FEC_EVENT              datetime64[ns]
PAGE                           object
CONTENT_CATEGORY               object
SITE_ID                        object
ON_SITE_SEARCH_TERM            object
USER_ID                         int32
weekday                        object
schedule                       object
tipo_dias                      object
trimestre                      object
dtype: object

In [4]:
sub = pd.read_csv("{}/sampleSubmission.zip".format(subfolder))
sub.head()

,USER_ID,SCORE
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0


In [6]:
sub.dtypes

USER_ID    int64
SCORE      int64
dtype: object

In [7]:
sub.shape

(11676, 2)

In [13]:
set(pageview['USER_ID'].unique()) == set(sub['USER_ID'].unique())

True

In [14]:
set(~pageview[pageview['PAGE'].str.contains('none')]['USER_ID'].unique()) == set(sub['USER_ID'].unique())

False

In [18]:
col_user = 'USER_ID'
device_data = pd.read_csv(
    "{}/device_data.zip".format(subfolder), parse_dates=["FEC_EVENT"]
).sort_values(
    [col_user, "FEC_EVENT"]
)

device_data['USER_ID'] = device_data['USER_ID'].astype(int)
device_data['CONNECTION_SPEED'] = device_data['CONNECTION_SPEED'].astype(str)
device_data['IS_MOBILE_DEVICE'] = device_data['IS_MOBILE_DEVICE'].astype(str)

device_data.shape, device_data['FEC_EVENT'].min(), device_data['FEC_EVENT'].max()

((2871406, 4),
 Timestamp('2018-01-12 00:00:06'),
 Timestamp('2018-12-31 23:59:35'))

In [19]:
pageview.shape, pageview['FEC_EVENT'].min(), pageview['FEC_EVENT'].max()

((22870354, 10),
 Timestamp('2018-01-01 00:09:17'),
 Timestamp('2018-12-31 23:59:59'))

In [15]:
pageview.isnull().sum()

FEC_EVENT              0
PAGE                   0
CONTENT_CATEGORY       0
SITE_ID                0
ON_SITE_SEARCH_TERM    0
USER_ID                0
weekday                0
schedule               0
tipo_dias              0
trimestre              0
dtype: int64

In [16]:
device_data.isnull().sum()

FEC_EVENT           0
CONNECTION_SPEED    0
IS_MOBILE_DEVICE    0
USER_ID             0
dtype: int64

In [11]:
pageview.dtypes

FEC_EVENT              datetime64[ns]
PAGE                           object
CONTENT_CATEGORY               object
SITE_ID                        object
ON_SITE_SEARCH_TERM            object
USER_ID                         int32
weekday                        object
schedule                       object
tipo_dias                      object
trimestre                      object
dtype: object

In [12]:
device_data.dtypes

FEC_EVENT           datetime64[ns]
CONNECTION_SPEED            object
IS_MOBILE_DEVICE            object
USER_ID                      int32
dtype: object

In [13]:
for col in pageview.columns:
    print(col, ': ', pageview[col].unique().shape[0])
print("/"*50)

for col in device_data.columns:
    print(col, ': ',device_data[col].unique().shape[0])

FEC_EVENT :  12397072
PAGE :  49150
CONTENT_CATEGORY :  65
SITE_ID :  4
ON_SITE_SEARCH_TERM :  294
USER_ID :  11676
weekday :  7
schedule :  6
tipo_dias :  4
trimestre :  4
//////////////////////////////////////////////////
FEC_EVENT :  2592597
CONNECTION_SPEED :  4
IS_MOBILE_DEVICE :  2
USER_ID :  11676


In [20]:
from IPython.display import display_html

def display_horizontal(*args, percent_sep=5):
    html_str=''
    for table in args:
        df = table if isinstance(table, pd.DataFrame) else  pd.DataFrame(table)
        html_str+=df.to_html()
    display_html(
        html_str.replace(
            'table','table style="display:inline;padding-right:{}%"'.format(percent_sep)
        ), 
        raw=True)

In [15]:
sufijo_view = 'view'
sufijo_device = 'device'
cols_select = ['PAGE', 'CONTENT_CATEGORY', 
               'CONTENT_CATEGORY_TOP', 'PAGE_PREVIOUS',
               'SITE_ID', 'ON_SITE_SEARCH_TERM',
               'CONNECTION_SPEED', 'IS_MOBILE_DEVICE',
               'weekday_{}'.format(sufijo_view), 
               'schedule_{}'.format(sufijo_view), 
               'day_cut_{}'.format(sufijo_view),
               'weekday_{}'.format(sufijo_device), 
               'schedule_{}'.format(sufijo_device), 
               'day_cut_{}'.format(sufijo_device)]

def get_trimestre(val):
    if val <= 3:
        return 1
    elif val <= 6:
        return 2
    elif val <= 9:
        return 3
    elif val <= 12:
        return 4
    return 0

def get_schedule(val):
    if val < 6:
        return 'madrugada'
    elif val < 13:
        return 'manana'
    elif val < 19:
        return 'tarde'
    else:
        return 'noche'

def get_day_cut(val):
    if val < 7:
        return 'inicio_mes'
    elif val < 13:
        return '7_12'
    elif val < 18:
        return 'quincena'
    elif val < 25:
        return '18_24'
    else:
        return 'fin_de_mes'
    
def add_schedule_weekday(data, sufijo=sufijo_view):
    data['hour'] = data['FEC_EVENT'].dt.hour
    data['day'] = data['FEC_EVENT'].dt.day
    data['weekday_{}'.format(sufijo_view)] = data['FEC_EVENT'].dt.weekday
    data['schedule_{}'.format(sufijo_view)] = data['hour'].apply(get_schedule)
    data['day_cut_{}'.format(sufijo_view)] = data['hour'].apply(get_day_cut)
    
    for col in ['CONTENT_CATEGORY_BOTTOM', 'hour', 'month', 'day']:
        if col in data.columns:
            del data[col]
    
    return data

def get_format_data(data):
    X_data = []
    
    for col in cols_select:
        if col in data.columns:
            print("-----> ", col)

            temp = pd.crosstab(
                data.USER_ID, data[col]
            )
            temp.columns = [col + "_" + str(v) for v in temp.columns]
            X_data.append(
                temp.apply(lambda x: x / x.sum(), axis=1)
            )

    return pd.concat(X_data, axis=1)



pageview['month'] = pageview['FEC_EVENT'].dt.month
pageview['trimestre'] = pageview['month'].apply(get_trimestre)

device_data['month'] = device_data['FEC_EVENT'].dt.month
device_data['trimestre'] = device_data['month'].apply(get_trimestre)

## del pageview['month']
pageview['trimestre'].unique(), device_data['trimestre'].unique()

(array([1, 2, 3, 4], dtype=int64), array([1, 2, 3, 4], dtype=int64))

In [16]:
# pageview[list(pageview.columns[1:])].corr()

In [17]:
if 'CONTENT_CATEGORY_BOTTOM' in pageview.columns:
    del pageview['CONTENT_CATEGORY_BOTTOM']

if 'month' in pageview.columns:
    del pageview['month']

if 'month' in device_data.columns:
    del device_data['month']

In [21]:
display_horizontal(
    pageview.groupby(['trimestre']).count()[col_user], 
    device_data.groupby(['trimestre']).count()[col_user]
)

,USER_ID
trimestre,
1,4861441
2,5677125
3,6154632
4,6177156
,USER_ID
trimestre,
1,565179
2,746963
3,781129


In [22]:
all_user = set(list(pageview[col_user].unique()))
len(all_user)

11676

In [ ]:
X_test = get_format_data(add_schedule_weekday(pageview.copy()))
X_test.shape

----->  PAGE


In [ ]:
X_device = get_format_data(add_schedule_weekday(device_data.copy(), sufijo=sufijo_device))
X_device.shape

In [ ]:
all_columns = list(X_test.columns)
X_test_mini = X_test[list(X_test.columns)[:2]].sort_index()
display(X_test_mini.head())

len(all_columns), type(X_test)

In [ ]:
data_view = []
cols_common_view = set()

for val in list(pageview['trimestre'].unique())[:-1]:
    print("/"*20, val, end=' ')
    page_trim = pageview[pageview['trimestre'] <= val]
    print(page_trim.shape, ' usuarios detectados: ',len(set(page_trim[col_user].unique())))

    # FORMAT DATA
    X_test_intern = get_format_data(add_schedule_weekday(page_trim))
    print("X_test_intern: ", X_test_intern.shape)
    
    # SELECT COMMON COLUMNSX_test_intern
    cols_common_view = set(X_test_intern.columns) if not cols_common_view else cols_common_view.intersection(set(X_test_intern.columns))
    print("cols_common_view:: ", len(cols_common_view))
    
    data_view.append(X_test_intern.copy())
    del X_test_intern
    del page_trim

# data_view = pd.concat(data_view, axis=0)
del pageview

In [ ]:
len(data_view)

In [ ]:

data_view = [_[list(cols_common_view)] for _ in data_view]

for dat in data_view:
    print(dat.shape)


## LOAD CONVERTIONS

In [ ]:
convertion = pd.read_csv(
    "{}/conversiones.csv".format(subfolder), dtype={col_user: int, 'mes': int}
).drop_duplicates()[['USER_ID','mes']].sort_values(
    ['USER_ID','mes']
).reset_index(drop=True)

display(convertion.head())
convertion.shape

In [ ]:
convertion['trimestre'] = convertion['mes'].apply(get_trimestre)
for col in convertion.columns:
    print(col, convertion[col].unique().shape)

del convertion['mes']
convertion.head()

In [ ]:
len(cols_common_view), type(cols_common_view)

In [ ]:
data_conv = []
col_target = 'TARGET'


def get_convertions(data_trim, data_analysis):
    users_data = data_analysis.index
    current = pd.Series(0, index=users_data)
    
    idx = set(
        data_trim.USER_ID.unique()
    ).intersection(
        set(users_data)
    )

    current.loc[list(idx)] = 1
    return current

for val in sorted(list(convertion['trimestre'].unique())):
    print("/"*20, val)
    conv_trim = convertion[convertion['trimestre'] == val].drop_duplicates().reset_index(drop=True)
    
    if val <= len(data_view):
        data_view[val-1]['current'] = get_convertions(conv_trim, data_view[val-1])
        
    elif val == len(data_view) + 1:
        X_test['current'] = get_convertions(conv_trim, X_test)
        
    if val > 1:
        data_view[val-2][col_target] = get_convertions(conv_trim, data_view[val-2])
            
    
cols_common_view.add('current')
len(cols_common_view)

In [ ]:
for dat in data_view:
    print(type(dat), dat.shape, col_target in dat.columns, 'current' in dat.columns)

In [ ]:
X_test.shape, 'current' in X_test.columns

In [ ]:
len(cols_common_view), 'current' in list(cols_common_view)

In [ ]:
cols_common_view = list(cols_common_view)
X_test = X_test[cols_common_view]
 
cols_common_view = cols_common_view + [col_target]

data = pd.concat(
    [_[cols_common_view] for _ in data_view], axis=0
)                    
data = data.drop_duplicates()
print("data:", data.shape)

X_train = data.drop([col_target], axis=1)
y_train = data[col_target]

del data
X_train.shape, X_test.shape, y_train.shape

In [ ]:
y_train.value_counts(dropna=False, normalize=True)

In [ ]:
def var_cols_relevants(sfm, X):
    bool_select_var = list(sfm.get_support())
    list_columns = list(X.columns)
    return list({
       list_columns[i]: bool_select_var[i] for i in range(len(list_columns)) if bool_select_var[i]
    }.keys())

## PREPARATE DATA

In [ ]:
from sklearn.linear_model import LassoCV
from sklearn.feature_selection import SelectFromModel
from lightgbm import LGBMClassifier
from sklearn import model_selection
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedShuffleSplit, StratifiedKFold, TimeSeriesSplit
from sklearn.model_selection import GridSearchCV
from sklearn import model_selection
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedShuffleSplit, StratifiedKFold, TimeSeriesSplit
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import TimeSeriesSplit, cross_val_score, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.feature_selection import VarianceThreshold
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.neighbors import KNeighborsClassifier

kfold_off = StratifiedKFold(
    n_splits=10, 
    shuffle=False, 
    random_state=SEED
)
time_split = TimeSeriesSplit(
    n_splits=10
)

"""
lasso_cv = LassoCV(cv=kfold_off, random_state=SEED, verbose=1, n_jobs=4)
sfm = SelectFromModel(lasso_cv)
sfm.fit(X_train, y_train)

list_var_relevants = var_cols_relevants(sfm, X_train)
print(list_var_relevants)
X_train = X_train[list_var_relevants]
X_test_k = X_test_k[list_var_relevants]
"""

In [ ]:
%%time
"""
logit = LogisticRegression(
    C=2.3263050671536263, 
    random_state=SEED, 
    solver='saga', 
    class_weight='balanced', 
    penalty='l2', 
    n_jobs = 4
)
cv_scores1 = cross_val_score(
    logit, X_train, y_train, cv=kfold_off, 
    scoring='roc_auc', n_jobs=4
)

print(cv_scores1)
print("/"*20, cv_scores1.mean())
"""

In [ ]:
%%time
"""
best_parameter = {
    'bagging_fraction': 0.95, 
    'boosting': 'dart', 
    'feature_fraction': 0.1, 
    'is_unbalance': True, 
    'learning_rate': 0.025, 
    'max_depth': 55, 
    'metric': 'auc', 
    'min_gain_to_split': 1, 
    'n_estimators': 500,
    'skip_drop': 0.5,
    'tree_learner': 'serial',
    'xgboost_dart_mode': False
}

model_lgbm =  LGBMClassifier(
    seed=SEED, **best_parameter
)
cv_scores2 = cross_val_score(
    model_lgbm, X_train, y_train, cv=kfold_off, 
    scoring='roc_auc', n_jobs=4
)

print(cv_scores2)
print("/"*20, cv_scores2.mean())
"""

In [ ]:
%%time
"""
model_knn = KNeighborsClassifier(
    n_neighbors=7,
    p=2, weights='distance'
)
cv_scores4 = cross_val_score(
    model_knn, X_train, y_train, cv=kfold_off, 
    scoring='roc_auc', n_jobs=4
)

print(cv_scores4)
print("/"*20, cv_scores4.mean())
"""

# INIT PREDICTIONS

In [ ]:
from sklearn.feature_selection import SelectKBest, chi2, f_classif
num_cols = [_*int(X_train.shape[1] / 12) for _ in range(1,12)]
num_cols[1:-2]

In [ ]:
del data_view

In [ ]:
import lightgbm as lgb

best_parameter = {
    'bagging_fraction': 0.95, 
    'boosting': 'dart', 
    'feature_fraction': 0.1, 
    'is_unbalance': True, 
    'learning_rate': 0.025, 
    'max_depth': 55, 
    'metric': 'auc', 
    'min_gain_to_split': 1, 
    'n_estimators': 500,
    'skip_drop': 0.5,
    'tree_learner': 'serial',
    'xgboost_dart_mode': False
}

param_grid = {
    k: [v] if not isinstance(v, list) else v for k, v in best_parameter.items()
}

fit_params = {
    "eval_metric" : 'auc', 
    'verbose': 2
}

def apply_lgbm(X_train_part, X_test_part, y_train):
    model_lgbm = lgb.LGBMClassifier(
        seed=SEED, 
        feature_fraction_seed=SEED, 
        drop_seed=SEED
    )
    grid = RandomizedSearchCV(
        model_lgbm, param_grid, cv=kfold_off, 
        n_jobs=-1, scoring='roc_auc', verbose=1,
        n_iter=100
    )
    grid.fit(X_train_part, y_train, **fit_params)
    
    # best_params_ = grid.best_params_
    # model_ = grid.best_estimator_
    # score_ = grid.best_score_

    print("-> ", grid.best_score_)
    
    predict_test = grid.best_estimator_.predict_proba(X_test_part)[:,1]
    predict_test = pd.DataFrame(
        {'USER_ID':index_test, 
         'SCORE':predict_test} 
    )
    display(predict_test.head())
    print(predict_test[predict_test['SCORE'] < 0.01].shape)
    print(predict_test[predict_test['SCORE'] > 0.5].shape)
    print(predict_test[predict_test['SCORE'] > 0.99].shape)
    
    predict_test.to_csv('lgbm_time_split_{}.csv'.format(intent), index=False)

In [ ]:
index_test = X_test.index

for num_col in num_cols[:-2]:
    sb = SelectKBest(f_classif, num_col)
    
    X_train_part = sb.fit_transform(X_train, y_train)
    X_test_part = sb.transform(X_test)

    
    X_test_part = sp.sparse.csr_matrix(X_test_part)
    X_train_part = sp.sparse.csr_matrix(X_train_part)
    print(">"*3, X_train_part.shape, X_test_part.shape, y_train.shape)
    
    intent = 'train_{}_rows_{}_cols'.format(*list(X_train_part.shape))
    print(intent)
    apply_lgbm(X_train_part, X_test_part, y_train)

### FORCE RAISE

In [ ]:
raise

In [ ]:
c_values = np.logspace(0, 0.75, 20)
c_values

In [ ]:
from sklearn.linear_model import LogisticRegression

param_log = {
    'C': c_values,
    'class_weight': ['balanced'],
    'solver': ['saga', 'newton-cg', 'sag'],
    'penalty': ['l2']
}

model_log = LogisticRegression(
    random_state=SEED, 
    n_jobs = 4
)

grid = GridSearchCV(
    model_log, param_log, cv=kfold_off, 
    n_jobs=-1, scoring='roc_auc', verbose=1
)
grid.fit(X_train, y_train)

best_params_ = grid.best_params_
model_ = grid.best_estimator_
score_ = grid.best_score_

print(score_, best_params_)

In [ ]:
predict_test_log = grid.best_estimator_.predict_proba(X_test_k)[:,1]   # 0.7757083940054187
predict_test_log, predict_test_log.shape

In [ ]:
predict_test_log = pd.DataFrame(
    {'USER_ID':index_test, 'SCORE':predict_test_log} 
)
predict_test_log.head()

In [ ]:
print(predict_test_log[predict_test_log['SCORE'] < 0.01].shape)
print(predict_test_log[predict_test_log['SCORE'] > 0.5].shape)
print(predict_test_log[predict_test_log['SCORE'] > 0.99].shape)
predict_test_log.to_csv('analysis_log_kof_{}x.csv'.format(intent), index=False)

In [ ]:
"""
(2033, 2)
(330, 2)
(8, 2)

(31, 2)
(6123, 2)
(153, 2)

(1641, 2)
(573, 2)
(297, 2)
"""

In [ ]:
gc.collect()